# Ingeniería Informática
# Programación declarativa @ URJC
# Prueba 1: programación funcional
## Curso 21-22, convocatoria extraordinaria (29 de junio de 2022)


# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

In [2]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Tests if the specified element belongs to this list
        def contains(a: A): Boolean 
 
        // Returns the number of elements of this list
        def length: Int
        
        // Reverse the elementos of this list
        def reverse: List[A]
    }
}

defined object Signatures

In [3]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1
__(2 puntos)__


__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar los siguientes teoremas y/o deducciones de la lógica intuicionista: 

$⊢ (\neg p \vee q) \rightarrow (p \vee r) \rightarrow (q \vee r) $

$ \{ p \rightarrow \neg q \} ⊢ \neg (p \wedge q) $

In [4]:
def resolution_law_A[P, Q, R]: (Either[Not[P], Q], Either[P, R]) => Either[Q, R] = 
    {
        case (Left(np), Left(p)) => np(p)
        case (Right(q), _) => Left(q)
        case (_, Right(r)) => Right(r)
    }

defined function resolution_law_A

In [5]:
def proofA[P, Q](p1: P => Not[Q]): Not[(P, Q)] = 
    (pq: (P, Q)) => p1(pq._1)(pq._2)
    // alternativamente
    //{ case (p: P, q: Q) => p1(p)(q) }

defined function proofA

In [7]:
def proofB[P, Q](p1: P => Q): Not[(P, Not[Q])] = 
    (pq: (P, Not[Q])) => pq._2(p1(pq._1))

defined function proofB

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ (p \rightarrow q) \rightarrow \neg p \vee q $

Supóngase para ello que la ley del tercio excluso se cumple para las variables proposicionales $p$ y $q$, es decir, que las fórmulas  $p \vee \neg p$ y $q \vee \neg q$ pueden utilizarse como premisas.

In [8]:
// no hace falta la premisa Either[Q, Not[Q]]
def proof[P, Q](p1: P => Q)(emp: Either[P, Not[P]]): Either[Not[P], Q] = 
    emp match {
        case Right(np) => Left(np)
        case Left(p) => Right(p1(p))
    }

defined function proof

# Ejercicio 2 
__(2 puntos)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$ 1 + \mathrm{Nel} X \cong \mathrm{List} X$

donde $\mathrm{Nel} X$ representa el tipo de las listas no vacías (es decir, con al menos un elemento), y se define de la siguiente forma: 

$\mathrm{Nel} X = X * \mathrm{List} X$.

La implementación del tipo $\mathrm{Nel} X$ en Scala es la siguiente:

In [10]:
type Nel[A] = (A, List[A])

defined type Nel

__a) (1,5 puntos)__ Demuestra en Scala el isomorfismo $ 1 + \mathrm{Nel} X \cong \mathrm{List} X$ mediante la implementación de las siguientes funciones `from` y `to`: 

In [11]:
def from[A](nel: Option[Nel[A]]): List[A] = 
    nel match {
        case None => Nil
        case Some((head, tail)) => head :: tail
    }

defined function from

In [12]:
def to[A](list: List[A]): Option[Nel[A]] = 
    list match {
        case Nil => None
        case head :: tail => Some(head, tail)
    }

defined function to

__b) (0,5 puntos)__ Completa el siguiente test unitario para comprobar que las funciones `from` y `to` efectivamente implementan el isomorfismo requerido.

In [13]:
class IsoTest(
    from: Option[Nel[Int]] => List[Int], 
    to: List[Int] => Option[Nel[Int]]
) extends FlatSpec with Matchers{
    
    "from-to" should "work" in {
        // Añade tres aserciones del tipo `... shouldBe ...`
        from(to(Nil)) shouldBe Nil
        from(to(1 :: Nil)) shouldBe 1 :: Nil
        from(to(1 :: 2 :: Nil)) shouldBe 1 :: 2 :: Nil
    }
    
    "to-from" should "work" in {
        // Añade tres aserciones del tipo `... shouldBe ...`
        to(from(None)) shouldBe None
        to(from(Some((0, Nil)))) shouldBe Some((0, Nil))
        to(from(Some((0, List(1,2,3))))) shouldBe Some((0, List(1,2,3)))
        
    }
}

defined class IsoTest

In [14]:
run(new IsoTest(from[Int], to[Int]))

cmd12$Helper$IsoTest:
from-to
- should work
to-from
- should work


# Ejercicio 3
__(2 puntos)__

La función `removeAll` elimina todas las ocurrencias de un determinado elemento en una lista. A continuación se muestran distintos casos de prueba particularizados para una lista de enteros:

In [10]:
class TestRemoveAll(
    removeAll: (Int, List[Int]) => List[Int]
) extends FlatSpec with Matchers{
    "removeAll" should "work" in {
        removeAll(1, List()) shouldBe List()
        removeAll(1, List(1,1,1,1)) shouldBe List()
        removeAll(1, List(1,2,3)) shouldBe List(2,3)
        removeAll(1, List(2,1,3)) shouldBe List(2,3)
        removeAll(1, List(2,3,1)) shouldBe List(2,3)
        removeAll(1, List(2,1,3,1)) shouldBe List(2,3)
    }
}

defined class TestRemoveAll

__a) (1 punto)__ Implementa la función `removeAll` de manera recursiva.

In [12]:
def removeAll[A](a: A, list: List[A]): List[A] = 
    list match {
        case Nil => Nil
        case head :: tail if head == a => 
            removeAll(a, tail)
        case head :: tail => 
            head :: removeAll(a, tail)
    }

defined function removeAll

In [21]:
// Alternativamente, para mostrar más claramente la correspondencia con la solución mediante `foldRight`
def removeAll[A](a: A, list: List[A]): List[A] = 
    list match {
        case Nil => Nil
        case head :: tail => 
            val tailSol = removeAll(a, tail)
            if (head == a) tailSol
            else head :: tailSol
    }

defined function removeAll

In [22]:
run(new TestRemoveAll(removeAll))

cmd9$Helper$TestRemoveAll:
removeAll
- should work


__b) (1 punto)__ Implementa la función `removeAll` utilizando la función `foldRight`/`filter`.

In [27]:
def removeAll[A](a: A, list: List[A]): List[A] = 
    list.foldRight(List[A]()){
        case (`a`, tailSol) => tailSol
        // Alternativamente: 
        // case (head, tailSol) if head == a => tailSol
        case (head, tailSol) => head :: tailSol
    }

defined function removeAll

In [28]:
run(new TestRemoveAll(removeAll))

cmd9$Helper$TestRemoveAll:
removeAll
- should work


In [29]:
def removeAll[A](a: A, list: List[A]): List[A] = 
    list.filter(_ != a)

defined function removeAll

In [17]:
run(new TestRemoveAll(removeAll))

cmd9$Helper$TestRemoveAll:
removeAll
- should work


# Ejercicio 4
__(4 puntos)__

La función `distinct` elimina todas las ocurrencias de un determinado elemento en una lista, _excepto_ la primera de ellas. A continuación se muestran distintos casos de prueba particularizados para una lista de enteros:

In [20]:
class TestDistinct(
    distinct: List[Int] => List[Int]
) extends FlatSpec with Matchers{
    "distinct" should "work" in {
        distinct(List()) shouldBe List()
        distinct(List(1,2,3)) shouldBe List(1,2,3)
        distinct(List(1,1,1,1)) shouldBe List(1)
        distinct(List(1,1,2,2,3,3)) shouldBe List(1,2,3)
        distinct(List(1,2,3,1,2,3)) shouldBe List(1,2,3)
        distinct(List(1,2,1,3,3,2)) shouldBe List(1,2,3)
    }
}

defined class TestDistinct

__a) (1 punto)__ Implementa la función `distinct` de manera recursiva, utilizando la función `removeAll` descrita en otro ejercicio de este mismo examen.

In [21]:
def distinct[A](list: List[A]): List[A] = 
    list match {
        case Nil => Nil
        case head :: tail => 
            head :: distinct(removeAll(head, tail))
    }

defined function distinct

In [22]:
run(new TestDistinct(distinct))

cmd19$Helper$TestDistinct:
distinct
- should work


__b) (1 punto)__ Implementa la función `distinct` utilizando las funciones `foldRight` y `removeAll`.

In [23]:
def distinct[A](list: List[A]): List[A] = 
    list.foldRight(List[A]())(
        (head, tailSol) => head :: removeAll(head, tailSol)
    )

defined function distinct

In [24]:
run(new TestDistinct(distinct))

cmd19$Helper$TestDistinct:
distinct
- should work


__c) (1 punto)__ Implementa la función `distinct` mediante recursión final (o de cola).

In [25]:
def distinct[A](list: List[A]): List[A] = {
    def aux(acc: List[A], list: List[A]): List[A] = 
        list match {
            case Nil => acc
            case a :: tail if acc.contains(a) => 
                aux(acc, tail)
            case a :: tail => 
                aux(a :: acc, tail)
        }

    aux(List(), list).reverse
}

defined function distinct

In [26]:
run(new TestDistinct(distinct))

cmd19$Helper$TestDistinct:
distinct
- should work


__d) (1 punto)__ Implementa la función `distinct` mediante la función `foldLeft`.

In [27]:
def distinct[A](list: List[A]): List[A] = 
    list.foldLeft(List[A]()){
        case (acc, a) if acc.contains(a) => acc
        case (acc, a) => a :: acc
    }.reverse

defined function distinct

In [28]:
run(new TestDistinct(distinct))

cmd19$Helper$TestDistinct:
distinct
- should work
